# Setup

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
import torch

In [2]:
class CFG:
    model = "Qwen/Qwen2-0.5B"

# Model

In [3]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model)
model = AutoModelForCausalLM.from_pretrained(CFG.model)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

# Zero-shot generalization

In [4]:
positive_logit, nagative_logit = tokenizer.encode(" positive"), tokenizer.encode(" negative")
positive_logit, nagative_logit

([6785], [8225])

Ta linia kodu pokazuje kodowanie dwóch słów: "positive" i "negative" przez tokenizer, przy czym każde słowo zaczyna się od spacji (zauważ znak spacji przed każdym słowem). Tokenizer zamienia te słowa na sekwencje liczb, które reprezentują tokeny w słowniku modelu.

Spacja przed słowami nie jest **przypadkowa** - w wielu modelach językowych sposób tokenizacji może się różnić w zależności od tego, czy słowo znajduje się na początku tekstu, czy jest poprzedzone spacją. Dodanie spacji przed słowem często prowadzi do bardziej przewidywalnej tokenizacji.

In [28]:
def score(review):
    prompt = f"""Question: Is the following review positive or
    negative about the movie?
    Review: {review} Answer:"""
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    final_logits = model(input_ids).logits[0, -1]

    if final_logits[positive_logit] > final_logits[nagative_logit]:
        print("Positive")
    else:
        print("Negative")

Ta funkcja dokonuje analizy sentymentu (nastawienia emocjonalnego) recenzji filmowej. Przeanalizujmy jej działanie krok po kroku.

Funkcja score przyjmuje jeden parametr - review, czyli tekst recenzji do przeanalizowania. Następnie tworzy prompt (zapytanie) używając f-stringa, który łączy stały tekst pytania z przekazaną recenzją. Powstały prompt ma formę pytania: "Czy ta recenzja jest pozytywna czy negatywna?" wraz z dołączoną do niego recenzją i słowem "Answer:" na końcu.

Następnie tekst jest przekształcany na formę zrozumiałą dla modelu poprzez tokenizację. Wywołanie tokenizer() z parametrem return_tensors="pt" oznacza, że wynik ma być zwrócony w formacie tensora PyTorch (stąd "pt"). Powstały tensor jest przechowywany w zmiennej input_ids.

Model przetwarza ten tensor wejściowy i generuje logits - są to surowe wartości przed ostatnią warstwą aktywacji. Interesuje nas ostatni token odpowiedzi ([0, -1] wybiera ostatni token z pierwszej sekwencji), ponieważ tam model powinien zdecydować o sentymencie.

Kluczowym elementem jest porównanie wartości logits dla dwóch konkretnych tokenów: 6785 i 8225. Te numery odpowiadają prawdopodobnie tokenom reprezentującym "positive" i "negative" w słowniku modelu. Jeśli wartość dla tokenu 6785 jest większa, funkcja uznaje recenzję za pozytywną, w przeciwnym razie za negatywną.

To podejście wykorzystuje fakt, że model został wytrenowany do przewidywania następnych tokenów w sekwencji. Po zobaczeniu pytania i recenzji, model przypisuje prawdopodobieństwa różnym możliwym odpowiedziom, a funkcja sprawdza, która z dwóch interesujących nas odpowiedzi otrzymała wyższe prawdopodobieństwo.

In [29]:
movie_review1 = """
It's so witless, in fact, that when we do discover the secret, we want to rewind the \
        film so we don't know the secret anymore.
        And then keep on rewinding, and rewinding, until we're back at the beginning, \
        and can get up from our seats and walk backward out of the theater and go down \
        the up escalator and watch the money spring from the cash register into our pockets.
        """

In [30]:
movie_review2 = """
I am required to award stars to movies I review. This time, I refuse to do it.
The star rating system is unsuited to this film. Is the movie good? Is it bad? Does it matter?
It is what it is and occupies a world where the stars don't shine.
        """

In [35]:
movie_review3 = """
"Pearl Harbor" is a two-hour movie squeezed into three hours, about how on Dec. 7, 1941, the Japanese staged a
surprise attack on an American love triangle. Its centerpiece is 40 minutes of redundant special effects,
surrounded by a love story of stunning banality. The film has been directed without grace, vision, or
originality, and although you may walk out quoting lines of dialog, it will not be because you admire them.
"""

In [32]:
score(movie_review1)

Negative


In [33]:
score(movie_review2)

Negative


In [34]:
score(movie_review3)

Negative
